# 3D Droplet Oscillation (Part 2, Postprocessing - Mode decomposition)

Results published: hopefully at some point!

This is part of the BoSSS-long-term validation test suite, which consists of several computationally expensive test-cases (runtime in the order of days), which are performed on a regular basis in order to validate the physical correctness of BoSSS simulations.

### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'D:\local\OscillatingDroplet3D'.


In [ ]:
//wmg.Sessions

In [ ]:
int numSess = wmg.Sessions.Count();
numSess

43

In [ ]:
//add database manually:
//OpenOrCreateDatabase(@"\\dc1\userspace\smuda\Databases\OscillatingDroplet3D");
OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D");
//OpenOrCreateDatabase(@"\\130.83.248.207\ValidationTests\OscillatingDroplet3D");

Opening existing database '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D'.


## Observing sessions

In [ ]:
var workSess = wmg.Sessions.Skip(numSess);
workSess

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_restart2*	05/30/2022 08:14:52	5cdca669...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_restart2*	05/27/2022 20:59:40	87cde3b9...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_restart2*	05/27/2022 20:59:17	33d67916...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_thirdOrderInit_restart2*	05/27/2022 20:58:55	efea163a...
#4: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr1_m4_Oh056_eta005_restart2*	05/27/2022 20:58:32	91e4a6b9...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_restart2*	05/27/2022 20:58:11	7c6f2773...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_restart2*	05/27/2022 20:57:50	a2d5a18a...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_restart2*	05/27/2022 20:57:27	99a785fa...
#8: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_thirdOrderInit_restart2*	05/27/2022 20:57:05	7534accc...
#9: OscillatingDrop

In [ ]:
//workSess.Pick(0).Delete(true);

Session 87ef392c-1c5f-43a6-8738-01634830ab3c deleted.


In [ ]:
//var targetDB = databases.Pick(0);
//wmg.Sessions.Skip(numSess).Take(14).CopyAll(targetDB); //Move(targetDB); //MoveAll(targetDB);
// var delSess = workSess.Take(6);
//wmg.Sessions[0].Delete(true)
//wmg.Sessions[8].Timesteps.Count()
// delSess

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta015*	05/30/2022 09:49:51	03e7638e...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03*	05/30/2022 09:49:24	2fa067c1...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04*	05/30/2022 09:48:58	543c23bc...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta015_thirdOrderInit*	05/30/2022 09:48:34	2fc7f304...
#4: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03_thirdOrderInit*	05/30/2022 09:48:10	78f416af...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_thirdOrderInit*	05/30/2022 09:47:47	2ff0f605...


In [ ]:
// foreach(var dS in delSess) {
//     dS.Delete(true);
// }

Session 03e7638e-613a-4ae6-a22e-f57820722ff5 deleted.
Session 2fa067c1-ff54-4405-b737-df094e5b0be2 deleted.
Session 543c23bc-96a4-4cc2-a193-4ba9b2e522ba deleted.
Session 2fc7f304-64d1-4ce3-b2f3-99fae7f6c886 deleted.
Session 78f416af-c991-42cc-b791-ab5c6ed559b4 deleted.
Session 2ff0f605-e77f-43b8-854f-081aa8a14c14 deleted.


In [ ]:
var sess = wmg.Sessions.Skip(numSess).Pick(29);
sess

OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_restart1*	05/17/2022 08:52:06	3763b09d...

In [ ]:
sess.GetSessionDirectory()

\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\8e7bc526-ef62-478a-b4c8-05a91e24944c

In [ ]:
sess.DeployPath

\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2022Apr08_113220

In [ ]:
sess.Timesteps

#0:  { Time-step: 417.0; Physical time: 2.0849999999999778s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#1:  { Time-step: 417.1; Physical time: 2.0849999999999778s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#2:  { Time-step: 417; Physical time: 2.0849999999999778s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#3:  { Time-step: 418; Physical time: 2.0899999999999777s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#4:  { Time-step: 419; Physical time: 2.0949999999999775s; Fields: Phi, PhiDG, VelocityX

In [ ]:
sess.Timesteps.Skip(5).Every(10).Export().WithSupersampling(2).Do();

Starting export process... Data will be written to the directory: C:\Users\smuda\AppData\Local\BoSSS\plots\sessions\OscillatingDroplet3D__OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_restart1__3763b09d-e0d2-4e1b-ad44-d1675652c728


In [ ]:
using System.IO;

# Study Overview

In [ ]:
var studySess = workSess.Skip(0).Take(18);
studySess

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_restart2*	05/30/2022 08:14:52	5cdca669...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_restart2*	05/27/2022 20:59:40	87cde3b9...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_restart2*	05/27/2022 20:59:17	33d67916...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_thirdOrderInit_restart2*	05/27/2022 20:58:55	efea163a...
#4: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr1_m4_Oh056_eta005_restart2*	05/27/2022 20:58:32	91e4a6b9...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_restart2*	05/27/2022 20:58:11	7c6f2773...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_restart2*	05/27/2022 20:57:50	a2d5a18a...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_restart2*	05/27/2022 20:57:27	99a785fa...
#8: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_thirdOrderInit_restart2*	05/27/2022 20:57:05	7534accc...
#9: OscillatingDrop

In [ ]:
string[] caseS = new string[] { "m2_Oh01_eta04", "m2_Oh01_eta02", "m2_Oh01_eta01", "m3_Oh01_eta04", "m3_Oh01_eta03", "m3_Oh01_eta015", "m4_Oh01_eta04", "m4_Oh01_eta01", "m4_Oh056_eta005"};
string[] gridS = new string[] { "J686", "J432" };
bool[] ShouldAnaInit = new bool[] { true, false };
bool[] useNewton = new bool[] { false };

# Plotting of Spherical Harmonics

In [ ]:
bool skipEqualModes = false;
bool skipUnequalModes = true;
bool skipNonPoly = true; // false: plots all Legendgre functions, i.e. indices (*, *)
//                          true:  plots only Legendre polynomials, i.e. inices (*, 0)

In [ ]:
int[] m2cases = new int[] {0, 1, 2};
int[] m3cases = new int[] {3, 4, 5};
int[] m4cases = new int[] {6, 7, 8};

In [ ]:
List<Plot2Ddata>[] PlotTableSH = new List<Plot2Ddata>[caseS.Length];
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases 
for(int iGrd = 0; iGrd < gridS.Length; iGrd++) {
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {
  
    string _case = caseS[iRow];
    string _grd = gridS[iGrd];

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_grd)
                                      && (sess.Name.Contains("_thirdOrderInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    if(SIs.IsNullOrEmpty()) 
        continue;

    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    Stack<ISessionInfo>  procSIs = new Stack<ISessionInfo>();
    procSIs.Push(SI);
    var currSI = SI;
    var rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    while(!rSIs.IsNullOrEmpty()) {
        var rSI = rSIs.Single();
        procSIs.Push(rSI);
        currSI = rSI;
        rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    }
    //Console.WriteLine("number of processed Sessions for {0}: {1}", _case, procSIs.Count);


    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3; 
    string description = " "+_grd;
    if (!ShouldAnaInit[iAna]) {
        fmt.DashType = DashTypes.Solid;
    } else {
        fmt.DashType = DashTypes.Dashed;
        description = description + "-3OrdInit";
    }
    if (useNewton[iNew]) {
        fmt.LineColor = LineColors.Red;
        description = description + "-Newton";
    } else {  
        fmt.LineColor = LineColors.Blue;
    }

    
    foreach(var pSI in procSIs) {

        //Console.WriteLine("processing Session {0}", pSI.Name);

        IDictionary<string, IList<double>> tab = null;
        try {
            tab = pSI.ReadTabulatedTextFileAsDoubles("SphericalHarmonics.txt", '\t');
            if (PlotTableSH[iRow] == null) 
                PlotTableSH[iRow] = new List<Plot2Ddata>();
        } catch (Exception e) {
            Console.Error.WriteLine(e.Message);
            Console.WriteLine("Skipping " + SI);
            
            //var dummyPlot = new Plot2Ddata();
            //dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
            //PlotTableSH[iRow] = new List<Plot2Ddata>();
            continue;
        }

        List<double> time = tab["time"].ToList();

        int cnt = 0;
        foreach(var column in tab) {

            //Console.WriteLine("processing column {0}", column.Key);
            
            if(column.Key == "time")
                continue;

            if(skipNonPoly) {
                if(!column.Key.Contains(", 0)")) {
                    continue;
                }
            }

            if(!m3cases.Contains(iRow)) { 
                if(column.Key.Contains("(1, 0)") || column.Key.Contains("(3, 0)") 
                || column.Key.Contains("(5, 0)") || column.Key.Contains("(7, 0)") 
                || column.Key.Contains("(9, 0)") || column.Key.Contains("(11, 0)")) {
                    continue;
                }
            }
            
            var plot = new Plot2Ddata();
            plot.ShowLegend = true;
            plot.Ylabel = column.Key;
            plot.XrangeMin = 0;
            plot.XrangeMax = 7;        

            string name = column.Key + description;    
    
            if(name.Contains("(0, 0)")) {
                List<double> procVal = new List<double>();
                foreach(double val in column.Value) {
                    procVal.Add(val-1);
                }
                plot.AddDataGroup(name, time, procVal, fmt);
            } else {   
                plot.AddDataGroup(name, time, column.Value, fmt);
            }
    
    
            if(PlotTableSH[iRow].Count > cnt) {
                var plotKey = PlotTableSH[iRow].ElementAt(cnt);
                var existingDatGrp = plotKey.dataGroups.Where(datgrp => datgrp.Name.Equals(name));
                if(existingDatGrp.IsNullOrEmpty()) {
                    //Console.WriteLine("no matching datagroup - add new group");
                    plotKey.AddDataGroup(name, plot.dataGroups[0].Abscissas, plot.dataGroups[0].Values, fmt);
                } else {
                    //Console.WriteLine("matching datagroup - add to existing group");
                    var datGrp = existingDatGrp.Single();
                    datGrp.Abscissas = datGrp.Abscissas.Concat(plot.dataGroups[0].Abscissas).ToArray();
                    datGrp.Values = datGrp.Values.Concat(plot.dataGroups[0].Values).ToArray();
                }
            } else {
                //Console.WriteLine("no matching datagroup - add new group");
                PlotTableSH[iRow].Add(plot);
            }
            cnt++;
    
       }

    }
   
}
}
}
}

OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_restart2
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_thirdOrderInit_restart2
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_restart2
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_thirdOrderInit_restart2
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_restart2
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta015_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta015
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_restart2
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_thirdOrderInit_restart2
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_restart2
OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_thirdOrderInit_restart2
OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_restart2
OD3D_J432k3_wallBC_amr1_m4_Oh056_eta005_thirdOrderInit_restart2
OD3D_J432k3_wallBC_amr1_m4_Oh056_eta005_restart2


In [ ]:
PlotTableSH[3]

index dataGroups LogX LogY LogBaseX LogBaseY LogX2 LogY2 XrangeMin XrangeMax YrangeMin YrangeMax Y2rangeMin Y2rangeMax X2rangeMin X2rangeMax Xlabel X2label Ylabel Y2label Title .. 0 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> (0, 0) <null> <null> 1 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> (1, 0) <null> <null> 2 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> (2, 0) <null> <null> 3 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> (3, 0) <null> <null> 4 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> (4, 0) <null> <null> 5 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> (5, 0) <null> <null> 6 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> (6, 0) <null> <null> 7 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> (7, 0) <null> <null> 8 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> (8, 0) <null> <null>

In [ ]:
int[] PlotTableSH_Count = new int[] {5, 5, 5, 8, 0, 8, 5, 5, 5};

### comparison data WNLT

In [ ]:
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
for(int i = 0; i < PlotTableSH_Count[iRow]; i++) {
    //Console.WriteLine(PlotTableSH[iRow].ElementAt(i).dataGroups[0].Name);
    MultidimensionalArray dat = MultidimensionalArray.Create(1);
    if(m2cases.Contains(iRow)) {
        try {
            dat = IMatrixExtensions.LoadFromTextFile($"data/ModeDecomposition/m2/modeDecomp{2*i}_{caseS[iRow]}.txt");
        } catch {
            continue;
        }
    } 
    if(m3cases.Contains(iRow)) {
        try {
            dat = IMatrixExtensions.LoadFromTextFile($"data/ModeDecomposition/m3/modeDecomp{i}_{caseS[iRow]}.txt");
        } catch {
            continue;
        }
    } 
    if(m4cases.Contains(iRow)) {
        try {
            dat = IMatrixExtensions.LoadFromTextFile($"data/ModeDecomposition/m4/modeDecomp{2*i}_{caseS[iRow]}.txt");
        } catch {
            continue;
        }
    } 

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    fmt.DashType = DashTypes.Dashed;
    fmt.LineColor = LineColors.Black;
    if(m3cases.Contains(iRow)) {
        PlotTableSH[iRow].ElementAt(i).AddDataGroup($"({i}, 0)-WNLT", dat.GetColumn(0), dat.GetColumn(1), fmt);
    } else {
        PlotTableSH[iRow].ElementAt(i).AddDataGroup($"({2*i}, 0)-WNLT", dat.GetColumn(0), dat.GetColumn(1), fmt);
    }
}
}

## Case 1 (m2_Oh01_eta04)

In [ ]:
var gp = PlotTableSH[0].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.035 
 
 
 
 
 -0.03 
 
 
 
 
 -0.025 
 
 
 
 
 -0.02 
 
 
 
 
 -0.015 
 
 
 
 
 -0.01 
 
 
 
 
 -0.005 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) J686 
 
 
 
 
 (0, 0) J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M1537.2,60.1 L1590.6,60.1 M106.4,251.2 L107.6,251.2 L108.7,251.2 L109.9,251.1 L111.0,250.9 L112.2,250.7
 L113.4,250.4 L114.5,250.0 L115.7,249.6 L116.9,249.2 L118.0,248.7 L119.2,248.2 L120.3,247.6 L121.5,246.9
 L122.7,246.3 L123.8,245.5 L125.0,244.8 L126.1,244.0 L127.3,243.1 L128.5,242.3 L129.6,241.3 L130.8,240.4
 L131.9,239.4 L133.1,238.4 L134.3,237.3 L135.4,236.2 L136.6,235.1 L137.8,234.0 L138.9,232.8 L140.1,231.6
 L141.2,230.4 L142.4,229.1 L143.6,227.8 L144.7,226.5 L145.9,225.2 L147.0,223.8 L148.2,222.5 L149.4,221.1
 L150.5,219.6 L151.7,218.2 L152.8,216.8 L154.0,215.3 L155.2,213.8 L156.3,212.3 L157.5,210.8 L158.7,209.2
 L159.8,207.7 L161.0,206.1 L162.1,204.5 L163.3,202.9 L164.5,201.3 L165.6,199.7 L166.8,198.1 L167.9,196.5
 L169.1,194.8 L170.3,193.1 L171.4,191.5 L172.6,189.8 L173.8,188.1 L174.9,186.4 L176.1,184.7 L177.2,183.0
 L178.4,181.3 L179.6,179.6 L180.7,177.9 L181.9,176.2 L183.0,174.5 L184.2,172.8 L185.4,171.0 L186.5,169.3
 L187.7,167.6 L188.8,165.9 L190.0,164.1 L191.2,162.4 L192.3,160.7 L193.5,159.0 L194.7,157.3 L195.8,155.6
 L197.0,153.8 L198.1,152.1 L199.3,150.4 L200.5,148.7 L201.6,147.0 L202.8,145.3 L203.9,143.6 L205.1,141.9
 L206.3,140.3 L207.4,138.6 L208.6,136.9 L209.7,135.3 L210.9,133.6 L212.1,132.0 L213.2,130.4 L214.4,128.8
 L215.6,127.1 L216.7,125.5 L217.9,123.9 L219.0,122.4 L220.2,120.8 L221.4,119.3 L222.5,117.7 L223.7,116.2
 L224.8,114.7 L226.0,113.2 L227.2,111.7 L228.3,110.2 L229.5,108.8 L230.6,107.4 L231.8,105.9 L233.0,104.5
 L234.1,103.1 L235.3,101.8 L236.5,100.4 L237.6,99.1 L238.8,97.8 L239.9,96.5 L241.1,95.3 L242.3,94.0
 L243.4,92.8 L244.6,91.6 L245.7,90.4 L246.9,89.3 L248.1,88.1 L249.2,87.0 L250.4,86.0 L251.6,84.9
 L252.7,83.9 L253.9,82.8 L255.0,81.9 L256.2,80.9 L257.4,80.0 L258.5,79.1 L259.7,78.2 L260.8,77.4
 L262.0,76.6 L263.2,75.8 L264.3,75.0 L265.5,74.3 L266.6,73.6 L267.8,72.9 L269.0,72.3 L270.1,71.7
 L271.3,71.1 L272.5,70.6 L273.6,70.1 L274.8,69.6 L275.9,69.1 L277.1,68.7 L278.3,68.3 L279.4,68.0
 L280.6,67.7 L281.7,67.4 L282.9,67.2 L284.1,66.9 L285.2,66.8 L286.4,66.6 L287.5,66.5 L288.7,66.4
 L289.9,66.4 L291.0,66.4 L292.2,66.4 L293.4,66.5 L294.5,66.6 L295.7,66.7 L296.8,66.8 L298.0,67.0
 L299.2,67.2 L300.3,67.5 L301.5,67.8 L302.6,68.1 L303.8,68.4 L305.0,68.8 L306.1,69.2 L307.3,69.7
 L308.5,70.1 L309.6,70.6 L310.8,71.1 L311.9,71.6 L313.1,72.2 L314.3,72.8 L315.4,73.4 L316.6,74.0
 L317.7,74.7 L318.9,75.4 L320.1,76.1 L321.2,76.8 L322.4,77.5 L323.5,78.3 L324.7,79.1 L325.9,79.8
 L327.0,80.6 L328.2,81.4 L329.4,82.3 L330.5,83.1 L331.7,84.0 L332.8,84.8 L334.0,85.6 L335.2,86.5
 L336.3,87.3 L337.5,88.2 L338.6,89.0 L339.8,89.9 L341.0,90.7 L342.1,91.6 L343.3,92.4 L344.4,93.3
 L345.6,94.2 L346.8,95.0 L347.9,95.9 L349.1,96.8 L350.3,97.6 L351.4,98.4 L352.6,99.3 L353.7,100.1
 L354.9,100.9 L356.1,101.7 L357.2,102.5 L358.4,103.3 L359.5,104.1 L360.7,104.8 L361.9,105.6 L363.0,106.3
 L364.2,107.0 L365.4,107.7 L366.5,108.4 L367.7,109.1 L368.8,109.7 L370.0,110.4 L371.2,111.0 L372.3,111.6
 L373.5,112.2 L374.6,112.7 L375.8,113.2 L377.0,113.7 L378.1,114.2 L379.3,114.7 L380.4,115.1 L381.6,115.6
 L382.8,115.9 L383.9,116.3 L385.1,116.7 L386.3,117.0 L387.4,117.3 L388.6,117.6 L389.7,117.8 L390.9,118.0
 L392.1,118.2 L393.2,118.4 L394.4,118.5 L395.5,118.6 L396.7,

In [ ]:
foreach(var lst in PlotTableSH[0]) {
foreach(var grp in lst.dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/ModeDecomposition/m2/" + caseS[0] + " " + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}
}

## Case 2 (m2_Oh01_eta02)

In [ ]:
var gp = PlotTableSH[1].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.009 
 
 
 
 
 -0.008 
 
 
 
 
 -0.007 
 
 
 
 
 -0.006 
 
 
 
 
 -0.005 
 
 
 
 
 -0.004 
 
 
 
 
 -0.003 
 
 
 
 
 -0.002 
 
 
 
 
 -0.001 
 
 
 
 
 0 
 
 
 
 
 0.001 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) J686-3OrdInit 
 
 
 
 
 (0, 0) J686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M1437.3,60.1 L1490.7,60.1 M106.4,242.3 L107.6,242.3 L108.7,242.3 L109.9,242.3 L111.0,242.2 L112.2,242.1
 L113.4,241.9 L114.5,241.6 L115.7,241.3 L116.9,241.0 L118.0,240.6 L119.2,240.1 L120.3,239.6 L121.5,239.0
 L122.7,238.3 L123.8,237.7 L125.0,237.0 L126.1,236.2 L127.3,235.4 L128.5,234.5 L129.6,233.6 L130.8,232.6
 L131.9,231.6 L133.1,230.6 L134.3,229.5 L135.4,228.4 L136.6,227.2 L137.8,226.0 L138.9,224.8 L140.1,223.5
 L141.2,222.2 L142.4,220.8 L143.6,219.4 L144.7,218.0 L145.9,216.6 L147.0,215.1 L148.2,213.6 L149.4,212.0
 L150.5,210.4 L151.7,208.8 L152.8,207.2 L154.0,205.5 L155.2,203.8 L156.3,202.1 L157.5,200.4 L158.7,198.7
 L159.8,196.9 L161.0,195.0 L162.1,193.3 L163.3,191.4 L164.5,189.6 L165.6,187.7 L166.8,185.8 L167.9,183.9
 L169.1,182.0 L170.3,180.1 L171.4,178.1 L172.6,176.2 L173.8,174.2 L174.9,172.2 L176.1,170.2 L177.2,168.2
 L178.4,166.2 L179.6,164.2 L180.7,162.2 L181.9,160.2 L183.0,158.2 L184.2,156.2 L185.4,154.2 L186.5,152.1
 L187.7,150.1 L188.8,148.1 L190.0,146.1 L191.2,144.1 L192.3,142.1 L193.5,140.1 L194.7,138.1 L195.8,136.1
 L197.0,134.1 L198.1,132.1 L199.3,130.2 L200.5,128.2 L201.6,126.2 L202.8,124.3 L203.9,122.4 L205.1,120.5
 L206.3,118.7 L207.4,116.7 L208.6,114.9 L209.7,113.1 L210.9,111.3 L212.1,109.5 L213.2,107.8 L214.4,106.0
 L215.6,104.3 L216.7,102.6 L217.9,101.0 L219.0,99.3 L220.2,97.7 L221.4,96.0 L222.5,94.5 L223.7,93.0
 L224.8,91.5 L226.0,90.0 L227.2,88.5 L228.3,87.1 L229.5,85.7 L230.6,84.4 L231.8,83.1 L233.0,81.7
 L234.1,80.5 L235.3,79.3 L236.5,78.1 L237.6,76.9 L238.8,75.9 L239.9,74.8 L241.1,73.8 L242.3,72.8
 L243.4,71.8 L244.6,70.9 L245.7,70.0 L246.9,69.1 L248.1,68.3 L249.2,67.5 L250.4,66.8 L251.6,66.1
 L252.7,65.5 L253.9,64.9 L255.0,64.3 L256.2,63.8 L257.4,63.3 L258.5,62.8 L259.7,62.4 L260.8,62.0
 L262.0,61.7 L263.2,61.4 L264.3,61.1 L265.5,60.9 L266.6,60.8 L267.8,60.6 L269.0,60.5 L270.1,60.5
 L271.3,60.5 L272.5,60.5 L273.6,60.6 L274.8,60.7 L275.9,60.8 L277.1,61.0 L278.3,61.2 L279.4,61.4
 L280.6,61.7 L281.7,62.0 L282.9,62.4 L284.1,62.8 L285.2,63.2 L286.4,63.6 L287.5,64.1 L288.7,64.6
 L289.9,65.2 L291.0,65.8 L292.2,66.4 L293.4,67.0 L294.5,67.7 L295.7,68.4 L296.8,69.1 L298.0,69.8
 L299.2,70.6 L300.3,71.4 L301.5,72.2 L302.6,73.0 L303.8,73.8 L305.0,74.7 L306.1,75.6 L307.3,76.5
 L308.5,77.3 L309.6,78.3 L310.8,79.2 L311.9,80.2 L313.1,81.1 L314.3,82.1 L315.4,83.1 L316.6,84.0
 L317.7,85.0 L318.9,86.0 L320.1,87.0 L321.2,88.1 L322.4,89.1 L323.5,90.1 L324.7,91.1 L325.9,92.1
 L327.0,93.1 L328.2,94.1 L329.4,95.1 L330.5,96.1 L331.7,97.1 L332.8,98.1 L334.0,99.1 L335.2,100.1
 L336.3,101.0 L337.5,102.0 L338.6,102.9 L339.8,103.9 L341.0,104.8 L342.1,105.7 L343.3,106.6 L344.4,107.5
 L345.6,108.3 L346.8,109.2 L347.9,110.0 L349.1,110.8 L350.3,111.6 L351.4,112.3 L352.6,112.9 L353.7,113.7
 L354.9,114.4 L356.1,115.1 L357.2,115.7 L358.4,116.4 L359.5,117.0 L360.7,117.6 L361.9,118.2 L363.0,118.7
 L364.2,119.3 L365.4,119.7 L366.5,120.2 L367.7,120.6 L368.8,121.0 L370.0,121.4 L371.2,121.8 L372.3,122.1
 L373.5,122.4 L374.6,122.7 L375.8,122.9 L377.0,123.1 L378.1,123.3 L379.3,123.5 L380.4,123.6 L381.6,123.7
 L382.8,123.8 L383.9,123.8 L385.1,123.8 L386.3,123.8 L387.4,12

In [ ]:
foreach(var lst in PlotTableSH[1]) {
    foreach(var grp in lst.dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        //string name = "plotData/ModeDecomposition/case2/" + grp.Name + ".txt";
        string name = "plotData/ModeDecomposition/m2/" + caseS[1] + " " + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
    }

## Case 3 (m2_Oh01_eta01)

In [ ]:
var gp = PlotTableSH[2].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.0025 
 
 
 
 
 -0.002 
 
 
 
 
 -0.0015 
 
 
 
 
 -0.001 
 
 
 
 
 -0.0005 
 
 
 
 
 0 
 
 
 
 
 0.0005 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) J686-3OrdInit 
 
 
 
 
 (0, 0) J686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M1437.3,60.1 L1490.7,60.1 M114.7,225.5 L115.9,225.5 L117.0,225.4 L118.2,225.3 L119.3,225.2 L120.5,225.0
 L121.6,224.7 L122.8,224.4 L123.9,224.1 L125.1,223.7 L126.3,223.2 L127.4,222.7 L128.6,222.2 L129.7,221.6
 L130.9,221.0 L132.0,220.3 L133.2,219.5 L134.3,218.8 L135.5,218.0 L136.7,217.1 L137.8,216.2 L139.0,215.2
 L140.1,214.3 L141.3,213.2 L142.4,212.2 L143.6,211.1 L144.7,209.9 L145.9,208.8 L147.0,207.6 L148.2,206.3
 L149.4,205.0 L150.5,203.7 L151.7,202.4 L152.8,201.0 L154.0,199.6 L155.1,198.2 L156.3,196.7 L157.4,195.2
 L158.6,193.7 L159.8,192.2 L160.9,190.6 L162.1,189.0 L163.2,187.4 L164.4,185.8 L165.5,184.1 L166.7,182.5
 L167.8,180.7 L169.0,179.0 L170.2,177.3 L171.3,175.6 L172.5,173.9 L173.6,172.1 L174.8,170.4 L175.9,168.5
 L177.1,166.8 L178.2,165.0 L179.4,163.2 L180.6,161.4 L181.7,159.5 L182.9,157.8 L184.0,156.0 L185.2,154.2
 L186.3,152.3 L187.5,150.5 L188.6,148.7 L189.8,146.9 L190.9,145.1 L192.1,143.3 L193.3,141.5 L194.4,139.7
 L195.6,138.0 L196.7,136.2 L197.9,134.5 L199.0,132.7 L200.2,131.0 L201.3,129.3 L202.5,127.6 L203.7,125.9
 L204.8,124.2 L206.0,122.5 L207.1,120.9 L208.3,119.3 L209.4,117.6 L210.6,116.1 L211.7,114.5 L212.9,113.0
 L214.1,111.5 L215.2,110.0 L216.4,108.5 L217.5,107.1 L218.7,105.6 L219.8,104.2 L221.0,102.9 L222.1,101.5
 L223.3,100.2 L224.5,98.8 L225.6,97.7 L226.8,96.5 L227.9,95.3 L229.1,94.1 L230.2,93.0 L231.4,91.9
 L232.5,90.8 L233.7,89.8 L234.8,88.7 L236.0,87.8 L237.2,86.8 L238.3,85.9 L239.5,85.1 L240.6,84.2
 L241.8,83.4 L242.9,82.7 L244.1,81.9 L245.2,81.2 L246.4,80.6 L247.6,79.9 L248.7,79.4 L249.9,78.8
 L251.0,78.3 L252.2,77.8 L253.3,77.4 L254.5,77.0 L255.6,76.6 L256.8,76.2 L258.0,75.9 L259.1,75.7
 L260.3,75.4 L261.4,75.3 L262.6,75.1 L263.7,75.0 L264.9,74.9 L266.0,74.8 L267.2,74.8 L268.4,74.8
 L269.5,74.9 L270.7,74.9 L271.8,75.1 L273.0,75.2 L274.1,75.4 L275.3,75.6 L276.4,75.8 L277.6,76.1
 L278.8,76.4 L279.9,76.7 L281.1,77.1 L282.2,77.5 L283.4,77.9 L284.5,78.3 L285.7,78.8 L286.8,79.3
 L288.0,79.7 L289.1,80.3 L290.3,80.9 L291.5,81.5 L292.6,82.1 L293.8,82.7 L294.9,83.4 L296.1,84.1
 L297.2,84.7 L298.4,85.2 L299.5,86.0 L300.7,86.8 L301.9,87.6 L303.0,88.2 L304.2,89.0 L305.3,89.9
 L306.5,90.7 L307.6,91.5 L308.8,92.3 L309.9,93.2 L311.1,94.0 L312.3,94.9 L313.4,95.7 L314.6,96.6
 L315.7,97.5 L316.9,98.3 L318.0,99.2 L319.2,100.1 L320.3,101.0 L321.5,101.9 L322.7,102.7 L323.8,103.6
 L325.0,104.5 L326.1,105.4 L327.3,106.2 L328.4,107.1 L329.6,107.9 L330.7,108.8 L331.9,109.6 L333.0,110.5
 L334.2,111.3 L335.4,112.1 L336.5,112.9 L337.7,113.7 L338.8,114.5 L340.0,115.3 L341.1,116.1 L342.3,116.8
 L343.4,117.6 L344.6,118.3 L345.8,119.0 L346.9,119.7 L348.1,120.3 L349.2,121.0 L350.4,121.6 L351.5,122.3
 L352.7,122.9 L353.8,123.5 L355.0,124.0 L356.2,124.6 L357.3,125.1 L358.5,125.6 L359.6,126.1 L360.8,126.6
 L361.9,127.0 L363.1,127.4 L364.2,127.8 L365.4,128.2 L366.6,128.4 L367.7,128.8 L368.9,129.1 L370.0,129.4
 L371.2,129.7 L372.3,129.9 L373.5,130.1 L374.6,130.3 L375.8,130.5 L376.9,130.6 L378.1,130.8 L379.3,130.9
 L380.4,130.9 L381.6,131.0 L382.7,131.0 L383.9,131.0 L385.0,131.0 L386.2,131.0 L387.3,130.9 L388.5,130.8
 L389.7,130.7 L390.8,130.6 L392.0,130.4 L393.1,130.2 L394.3,130.0 L395.4,129.8 L396.6,129.6 L397.7,129.3
 L398.9,129.0 L400.1,128.7 L401.2,128.4 L40

In [ ]:
foreach(var lst in PlotTableSH[2]) {
    foreach(var grp in lst.dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/ModeDecomposition/m2/" + caseS[2] + " " + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}

## Case 4 (m3_Oh01_eta04)

In [ ]:
var gp = PlotTableSH[3].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at BoSSS.Solution.Gnuplot.GnuplotExtensions.ToGnuplot(List`1 _2DData, Int32[,] layout) in D:\BoSSS-experimental\public\src\L3-solution\BoSSS.Solution.Gnuplot\GnuplotExtensions.cs:line 346
   at Submission#102.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
foreach(var lst in PlotTableSH[3]) {
    foreach(var grp in lst.dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/ModeDecomposition/case4/" + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
    }

## Case 7 (m4_Oh01_eta04)

In [ ]:
var gp = PlotTableSH[6].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.02 
 
 
 
 
 -0.018 
 
 
 
 
 -0.016 
 
 
 
 
 -0.014 
 
 
 
 
 -0.012 
 
 
 
 
 -0.01 
 
 
 
 
 -0.008 
 
 
 
 
 -0.006 
 
 
 
 
 -0.004 
 
 
 
 
 -0.002 
 
 
 
 
 0 
 
 
 
 
 0.002 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) J686 
 
 
 
 
 (0, 0) J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M1537.2,60.1 L1590.6,60.1 M106.4,242.6 L107.6,242.6 L108.7,242.2 L109.9,241.5 L111.0,240.3 L112.2,238.7
 L113.4,236.8 L114.5,234.6 L115.7,232.0 L116.9,229.1 L118.0,225.8 L119.2,222.4 L120.3,218.5 L121.5,214.6
 L122.7,210.5 L123.8,206.1 L125.0,201.6 L126.1,197.0 L127.3,192.3 L128.5,187.4 L129.6,182.5 L130.8,177.6
 L131.9,172.5 L133.1,167.5 L134.3,162.5 L135.4,157.6 L136.6,152.6 L137.8,147.7 L138.9,142.9 L140.1,138.2
 L141.2,133.5 L142.4,129.0 L143.6,124.6 L144.7,120.3 L145.9,116.2 L147.0,112.3 L148.2,108.5 L149.4,105.0
 L150.5,101.6 L151.7,98.6 L152.8,95.6 L154.0,92.7 L155.2,90.0 L156.3,87.4 L157.5,85.1 L158.7,82.9
 L159.8,80.9 L161.0,79.0 L162.1,77.3 L163.3,75.7 L164.5,74.4 L165.6,73.2 L166.8,72.3 L167.9,71.6
 L169.1,71.0 L170.3,70.5 L171.4,70.4 L172.6,70.6 L173.8,70.8 L174.9,70.9 L176.1,71.3 L177.2,71.9
 L178.4,72.5 L179.6,73.2 L180.7,74.0 L181.9,74.9 L183.0,76.0 L184.2,77.2 L185.4,78.4 L186.5,79.7
 L187.7,81.2 L188.8,82.6 L190.0,84.0 L191.2,85.5 L192.3,87.0 L193.5,88.5 L194.7,90.0 L195.8,91.4
 L197.0,92.6 L198.1,94.1 L199.3,95.4 L200.5,96.7 L201.6,97.6 L202.8,98.7 L203.9,99.9 L205.1,100.8
 L206.3,101.7 L207.4,102.4 L208.6,103.1 L209.7,103.6 L210.9,104.0 L212.1,104.3 L213.2,104.5 L214.4,104.7
 L215.6,104.7 L216.7,104.6 L217.9,104.4 L219.0,104.1 L220.2,103.7 L221.4,103.2 L222.5,102.7 L223.7,102.1
 L224.8,101.4 L226.0,100.0 L227.2,99.4 L228.3,98.6 L229.5,97.5 L230.6,96.7 L231.8,95.8 L233.0,94.9
 L234.1,93.9 L235.3,92.8 L236.5,91.8 L237.6,90.6 L238.8,89.6 L239.9,88.7 L241.1,87.6 L242.3,86.6
 L243.4,85.6 L244.6,84.7 L245.7,83.7 L246.9,82.8 L248.1,81.9 L249.2,81.0 L250.4,80.2 L251.6,79.3
 L252.7,78.6 L253.9,77.8 L255.0,77.0 L256.2,76.3 L257.4,75.7 L258.5,75.1 L259.7,74.6 L260.8,74.0
 L262.0,73.6 L263.2,73.2 L264.3,72.8 L265.5,72.4 L266.6,72.1 L267.8,71.9 L269.0,71.4 L270.1,71.3
 L271.3,71.2 L272.5,71.1 L273.6,71.0 L274.8,71.0 L275.9,70.9 L277.1,70.9 L278.3,70.8 L279.4,70.8
 L280.6,70.9 L281.7,70.9 L282.9,71.0 L284.1,71.0 L285.2,71.1 L286.4,71.2 L287.5,71.3 L288.7,71.5
 L289.9,71.5 L291.0,71.7 L292.2,71.7 L293.4,71.8 L294.5,71.8 L295.7,71.9 L296.8,71.9 L298.0,72.0
 L299.2,72.0 L300.3,72.0 L301.5,72.0 L302.6,72.0 L303.8,72.0 L305.0,72.0 L306.1,71.9 L307.3,71.8
 L308.5,71.7 L309.6,71.6 L310.8,71.5 L311.9,71.4 L313.1,71.2 L314.3,71.0 L315.4,70.8 L316.6,70.6
 L317.7,70.4 L318.9,70.2 L320.1,69.9 L321.2,69.6 L322.4,69.4 L323.5,69.1 L324.7,68.8 L325.9,68.5
 L327.0,68.2 L328.2,67.9 L329.4,67.5 L330.5,67.2 L331.7,66.9 L332.8,66.9 L334.0,66.5 L335.2,66.1
 L336.3,65.7 L337.5,65.4 L338.6,65.0 L339.8,64.9 L341.0,64.5 L342.1,64.1 L343.3,63.8 L344.4,63.5
 L345.6,63.2 L346.8,62.9 L347.9,62.7 L349.1,62.4 L350.3,62.2 L351.4,62.1 L352.6,61.8 L353.7,61.6
 L354.9,61.4 L356.1,61.2 L357.2,61.0 L358.4,60.9 L359.5,60.7 L360.7,60.6 L361.9,60.5 L363.0,60.4
 L364.2,60.3 L365.4,60.2 L366.5,60.1 L367.7,60.1 L368.8,60.0 L370.0,60.0 L371.2,59.9 L372.3,59.9
 L373.5,59.9 L374.6,59.9 L375.8,60.1 L377.0,60.1 L378.1,60.1 L379.3,60.3 L380.4,60.4 L381.6,60.5
 L382.8,60.5 L383.9,60.6 L385.1,60.6 L386.3,60.7 L387.4,60.7 L388.6,60.8 L389.7,60.8 L390.9,60.9
 L392.1,61.0 L393.2,60.9 L394.4,61.0 L395.5,61.1 L396.7,61.2 L397.

In [ ]:
foreach(var lst in PlotTableSH[6]) {
    foreach(var grp in lst.dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/ModeDecomposition/m4/" + caseS[6] + " " + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}

## Case 8 (m4_Oh01_eta01)

In [ ]:
var gp = PlotTableSH[7].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.0012 
 
 
 
 
 -0.001 
 
 
 
 
 -0.0008 
 
 
 
 
 -0.0006 
 
 
 
 
 -0.0004 
 
 
 
 
 -0.0002 
 
 
 
 
 0 
 
 
 
 
 0.0002 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) J686-3OrdInit 
 
 
 
 
 (0, 0) J686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M1437.3,60.1 L1490.7,60.1 M114.7,248.1 L115.9,248.5 L117.0,248.0 L118.2,247.1 L119.3,245.7 L120.5,243.8
 L121.6,241.4 L122.8,238.6 L123.9,235.4 L125.1,231.8 L126.3,227.9 L127.4,223.6 L128.6,219.0 L129.7,214.1
 L130.9,208.8 L132.0,203.5 L133.2,198.0 L134.3,192.3 L135.5,186.5 L136.7,180.7 L137.8,174.7 L139.0,168.7
 L140.1,162.7 L141.3,156.8 L142.4,150.9 L143.6,145.1 L144.7,139.4 L145.9,133.5 L147.0,128.2 L148.2,123.0
 L149.4,118.0 L150.5,113.2 L151.7,108.6 L152.8,104.3 L154.0,100.2 L155.1,96.3 L156.3,92.8 L157.4,89.5
 L158.6,86.4 L159.8,83.7 L160.9,81.2 L162.1,79.1 L163.2,77.2 L164.4,75.6 L165.5,74.3 L166.7,73.3
 L167.8,72.5 L169.0,72.0 L170.2,71.8 L171.3,71.8 L172.5,72.0 L173.6,72.4 L174.8,72.6 L175.9,73.7
 L177.1,74.8 L178.2,75.6 L179.4,77.2 L180.6,78.7 L181.7,80.2 L182.9,81.9 L184.0,83.6 L185.2,85.4
 L186.3,87.2 L187.5,89.0 L188.6,90.8 L189.8,92.6 L190.9,93.7 L192.1,95.7 L193.3,97.5 L194.4,99.1
 L195.6,100.7 L196.7,102.2 L197.9,103.6 L199.0,104.9 L200.2,106.1 L201.3,107.1 L202.5,108.1 L203.7,108.9
 L204.8,109.5 L206.0,110.1 L207.1,110.4 L208.3,110.7 L209.4,110.9 L210.6,110.9 L211.7,110.7 L212.9,110.5
 L214.1,110.1 L215.2,109.6 L216.4,109.0 L217.5,108.4 L218.7,107.6 L219.8,106.7 L221.0,105.7 L222.1,104.7
 L223.3,103.6 L224.5,102.4 L225.6,101.2 L226.8,100.0 L227.9,98.7 L229.1,97.4 L230.2,96.1 L231.4,94.7
 L232.5,93.0 L233.7,91.9 L234.8,90.7 L236.0,89.4 L237.2,88.1 L238.3,86.9 L239.5,85.7 L240.6,84.5
 L241.8,83.4 L242.9,82.3 L244.1,81.3 L245.2,80.3 L246.4,79.4 L247.6,78.5 L248.7,77.8 L249.9,77.0
 L251.0,76.4 L252.2,75.5 L253.3,75.0 L254.5,74.5 L255.6,74.1 L256.8,73.8 L258.0,73.5 L259.1,73.3
 L260.3,73.1 L261.4,73.1 L262.6,73.0 L263.7,73.0 L264.9,73.1 L266.0,73.2 L267.2,73.4 L268.4,73.6
 L269.5,73.8 L270.7,74.1 L271.8,74.4 L273.0,74.7 L274.1,75.0 L275.3,75.4 L276.4,75.8 L277.6,76.2
 L278.8,76.6 L279.9,77.0 L281.1,77.3 L282.2,77.7 L283.4,78.1 L284.5,78.5 L285.7,78.9 L286.8,79.2
 L288.0,79.5 L289.1,79.8 L290.3,80.1 L291.5,80.4 L292.6,80.6 L293.8,80.9 L294.9,81.1 L296.1,81.2
 L297.2,81.3 L298.4,81.5 L299.5,81.5 L300.7,81.6 L301.9,81.6 L303.0,81.6 L304.2,81.6 L305.3,81.5
 L306.5,81.4 L307.6,81.3 L308.8,81.1 L309.9,81.0 L311.1,80.8 L312.3,80.6 L313.4,80.4 L314.6,80.1
 L315.7,79.9 L316.9,79.6 L318.0,79.3 L319.2,79.0 L320.3,78.7 L321.5,78.4 L322.7,78.1 L323.8,77.8
 L325.0,77.5 L326.1,77.1 L327.3,76.8 L328.4,76.5 L329.6,76.2 L330.7,75.9 L331.9,75.6 L333.0,75.3
 L334.2,75.0 L335.4,74.7 L336.5,74.5 L337.7,74.2 L338.8,74.0 L340.0,73.8 L341.1,73.6 L342.3,73.4
 L343.4,73.2 L344.6,73.0 L345.8,72.9 L346.9,72.8 L348.1,72.7 L349.2,72.6 L350.4,72.5 L351.5,72.4
 L352.7,72.3 L353.8,72.3 L355.0,72.3 L356.2,72.3 L357.3,72.3 L358.5,72.3 L359.6,72.3 L360.8,72.3
 L361.9,72.4 L363.1,72.4 L364.2,72.5 L365.4,72.5 L366.6,72.6 L367.7,72.7 L368.9,72.8 L370.0,72.9
 L371.2,72.9 L372.3,73.0 L373.5,73.1 L374.6,73.2 L375.8,73.3 L376.9,73.4 L378.1,73.5 L379.3,73.6
 L380.4,73.6 L381.6,73.7 L382.7,73.8 L383.9,73.6 L385.0,73.7 L386.2,73.9 L387.3,74.0 L388.5,74.1
 L389.7,74.1 L390.8,74.2 L392.0,74.2 L393.1,74.2 L394.3,74.3 L395.4,74.3 L396.6,74.3 L397.7,74.3
 L398.9,74.3 L400.1,74.2 L401.2,74.2 L402.4,74.2 L403.5,74.2 L404.7,73.9 L405.8,73.8 L407.0,73.9
 L408.1,73.9 L409.

In [ ]:
foreach(var lst in PlotTableSH[7]) {
    foreach(var grp in lst.dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/ModeDecomposition/m4/" + caseS[7] + " " + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}

## Case 9 (m4_Oh056_eta005)

In [ ]:
var gp = PlotTableSH[8].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.0003 
 
 
 
 
 -0.00025 
 
 
 
 
 -0.0002 
 
 
 
 
 -0.00015 
 
 
 
 
 -0.0001 
 
 
 
 
 -5x10 -5 
 
 
 
 
 0 
 
 
 
 
 5x10 -5 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) J686-3OrdInit 
 
 
 
 
 (0, 0) J686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M1437.3,60.1 L1490.7,60.1 M123.0,247.9 L124.1,249.1 L125.3,249.4 L126.4,249.4 L127.6,249.1 L128.7,248.5
 L129.9,247.5 L131.0,246.3 L132.2,244.8 L133.3,243.1 L134.5,241.2 L135.6,239.2 L136.8,236.9 L137.9,234.5
 L139.1,232.0 L140.2,229.4 L141.4,226.8 L142.5,224.1 L143.7,221.3 L144.8,218.5 L146.0,215.7 L147.1,213.0
 L148.3,210.2 L149.4,207.4 L150.6,204.7 L151.7,202.1 L152.9,199.5 L154.0,196.9 L155.2,194.4 L156.3,192.0
 L157.5,189.7 L158.6,187.4 L159.8,185.2 L160.9,183.1 L162.1,181.1 L163.2,179.2 L164.4,177.4 L165.5,175.6
 L166.7,173.9 L167.8,172.4 L169.0,170.9 L170.1,169.4 L171.3,168.1 L172.4,166.8 L173.6,165.6 L174.7,164.5
 L175.9,163.5 L177.0,162.5 L178.2,161.6 L179.3,160.8 L180.5,160.0 L181.6,159.2 L182.8,158.5 L183.9,157.9
 L185.1,157.3 L186.2,156.8 L187.4,156.3 L188.5,155.9 L189.7,155.4 L190.8,155.1 L192.0,154.7 L193.1,154.4
 L194.3,154.1 L195.4,153.8 L196.6,153.6 L197.7,153.4 L198.9,153.1 L200.0,153.0 L201.2,152.8 L202.3,152.6
 L203.5,152.4 L204.6,152.3 L205.8,152.2 L206.9,152.0 L208.1,151.9 L209.2,151.7 L210.4,151.6 L211.5,151.5
 L212.6,151.3 L213.8,151.2 L214.9,151.0 L216.1,150.9 L217.2,150.7 L218.4,150.6 L219.5,150.4 L220.7,150.2
 L221.8,150.0 L223.0,149.8 L224.1,149.6 L225.3,149.4 L226.4,149.2 L227.6,149.0 L228.7,148.7 L229.9,148.5
 L231.0,148.2 L232.2,147.9 L233.3,147.6 L234.5,147.3 L235.6,147.0 L236.8,146.7 L237.9,146.4 L239.1,146.0
 L240.2,145.6 L241.4,145.3 L242.5,144.3 L243.7,144.1 L244.8,143.4 L246.0,143.2 L247.1,142.8 L248.3,142.4
 L249.4,142.0 L250.6,141.5 L251.7,141.1 L252.9,140.6 L254.0,140.1 L255.2,139.6 L256.3,139.1 L257.5,138.6
 L258.6,138.0 L259.8,137.5 L260.9,136.9 L262.1,136.4 L263.2,135.8 L264.4,135.2 L265.5,134.7 L266.7,134.1
 L267.8,133.5 L269.0,132.9 L270.1,132.3 L271.3,131.3 L272.4,131.0 L273.6,130.2 L274.7,129.9 L275.9,129.3
 L277.0,128.7 L278.2,128.1 L279.3,127.4 L280.5,126.8 L281.6,126.2 L282.8,125.5 L283.9,124.9 L285.1,124.2
 L286.2,123.6 L287.4,123.0 L288.5,122.3 L289.7,121.7 L290.8,121.0 L292.0,120.4 L293.1,119.7 L294.3,119.1
 L295.4,118.4 L296.6,117.8 L297.7,117.2 L298.9,116.5 L300.0,115.9 L301.2,115.3 L302.3,114.6 L303.4,114.0
 L304.6,113.4 L305.7,112.8 L306.9,112.2 L308.0,111.5 L309.2,110.9 L310.3,110.3 L311.5,109.7 L312.6,109.1
 L313.8,108.5 L314.9,108.0 L316.1,107.4 L317.2,106.8 L318.4,106.2 L319.5,105.7 L320.7,105.1 L321.8,104.6
 L323.0,104.0 L324.1,103.5 L325.3,102.9 L326.4,102.4 L327.6,101.9 L328.7,101.3 L329.9,100.8 L331.0,100.3
 L332.2,99.8 L333.3,99.3 L334.5,98.8 L335.6,98.4 L336.8,97.9 L337.9,97.4 L339.1,96.9 L340.2,96.5
 L341.4,96.0 L342.5,95.6 L343.7,95.1 L344.8,94.7 L346.0,94.3 L347.1,93.9 L348.3,93.5 L349.4,93.0
 L350.6,92.6 L351.7,92.3 L352.9,91.9 L354.0,91.5 L355.2,91.1 L356.3,90.7 L357.5,90.4 L358.6,90.0
 L359.8,89.7 L360.9,89.3 L362.1,89.0 L363.2,88.7 L364.4,88.3 L365.5,88.0 L366.7,87.7 L367.8,87.4
 L369.0,87.1 L370.1,86.8 L371.3,86.5 L372.4,86.2 L373.6,85.9 L374.7,85.6 L375.9,85.4 L377.0,85.1
 L378.2,84.9 L379.3,84.6 L380.5,84.4 L381.6,84.1 L382.8,83.9 L383.9,83.6 L385.1,83.4 L386.2,83.2
 L387.4,83.0 L388.5,82.8 L389.7,82.6 L390.8,82.4 L391.9,82.2 L393.1,82.0 L394.2,81.8 L395.4,81.6
 L396.5,81.4 L397.7,81.2 L398.8,81.1 L400.0,80.9 L401.1,80.7 L402.3,80.6 L403.4,80.4 L40

In [ ]:
foreach(var lst in PlotTableSH[8]) {
    foreach(var grp in lst.dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/ModeDecomposition/m4/" + caseS[8] + " " + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}